In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"
new_model = "llama-3.2-3b-chat-doctor"

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

# base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
messages = [{"role": "user", "content": "Hello doctor, I have bad acne. How do I get rid of it?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 27 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello doctor, I have bad acne. How do I get rid of it?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hi, I understand your concern. You should not worry about acne. It is very common and it can be treated. For better treatment you should consult a dermatologist. He will examine you and will advise you about the treatment accordingly. In addition to that you can also apply a benzoyl peroxide gel or a spot treatment containing salicylic acid or metronidazole. Avoid touching or popping pimples as it can worsen the situation. For more information consult a dermatologist online -->  -->  -->  -->  -->  -->  -->  -->  -->  -->  -->


In [7]:
model.save_pretrained("merged_llama-3.2-3b-chat-doctor")
tokenizer.save_pretrained("merged_llama-3.2-3b-chat-doctor")

('merged_llama-3.2-3b-chat-doctor/tokenizer_config.json',
 'merged_llama-3.2-3b-chat-doctor/special_tokens_map.json',
 'merged_llama-3.2-3b-chat-doctor/tokenizer.json')

In [ ]:
# model.push_to_hub("llama-3-8b-chat-doctor", use_temp_dir=False)
# tokenizer.push_to_hub("llama-3-8b-chat-doctor", use_temp_dir=False)